# data

In [3]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras import layers
from keras import optimizers

밑에서 사용할 Conv1DTranspose는 tensorflow 2.3이어야 가능

In [2]:
import tensorflow as tf
print(tf.version.VERSION)

2.3.0


train1.csv 만 불러왔음

In [4]:
train1_raw = pd.read_csv("data/HAI2.0/training/train1.csv")
train1_raw.head()

,time,C01,C02,C03,C04,C05,C06,C07,C08,C09,...,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79
0,2020-07-11 00:00:00,395.19528,12,10,52.80456,-1.2648,-1.87531,779.59595,28.02645,10832.0,...,808.29620,0.0,1.36810,8.79882,35.43700,12.01782,305.03113,301.35992,33.6555,6.0951
1,2020-07-11 00:00:01,395.14420,12,10,52.78931,-1.3147,-1.88294,780.67328,28.02473,10984.0,...,819.16809,0.0,1.36810,8.78811,35.45227,12.01782,304.27161,297.43567,33.6555,5.9262
2,2020-07-11 00:00:02,395.14420,12,10,52.79694,-1.4032,-1.88294,780.06574,28.02817,11120.0,...,823.51697,0.0,1.36734,8.81787,35.45227,12.01782,303.89179,298.66534,33.6555,5.8101
3,2020-07-11 00:00:03,395.19528,12,10,52.79694,-1.6074,-1.88294,780.15265,28.02301,11256.0,...,823.95172,0.0,1.36734,8.87493,35.43700,12.01782,303.67474,298.06860,33.6555,5.7509
4,2020-07-11 00:00:04,395.34866,12,10,52.79694,-1.7811,-1.88294,781.83160,28.03595,11384.0,...,827.86560,0.0,1.36810,8.83838,35.45227,12.01782,303.22266,296.53137,33.6555,5.8547


In [5]:
columns = train1_raw.columns.drop("time")
train_min = train1_raw[columns].min()
train_max = train1_raw[columns].max()

In [6]:
def normalize(df):
    ndf = df.copy()
    for c in df.columns:
        if train_min[c] == train_max[c]:
            ndf[c] = df[c] - train_min[c]
        else:
            ndf[c] = (df[c] - train_min[c]) / (train_max[c] - train_min[c])
    return ndf

def boundary_check(df):
    x = np.array(df, dtype=np.float32)
    return np.any(x > 1.0), np.any(x < 0), np.any(np.isnan(x))

In [7]:
train = normalize(train1_raw[columns])

In [8]:
boundary_check(train)

(False, False, False)

In [9]:
train_np = train.values
train_np.shape

(216001, 79)

In [10]:
ts = 90 #timestep

밑에 코드로 lstm에 넣는 데이터 형태로 바꿔줬는데
Colab에서 실행해보니까 RAM 용량땜에 안되네요... ㅠㅠ

In [9]:
x_train = np.zeros((train_np.shape[0]-ts,ts,train_np.shape[1]))

for i in range(x_train.shape[0]):
    for j in range(ts):
        x_train[i][j] = train_np[i+j]
    

In [11]:
x_train.shape

(215911, 90, 79)

다른방법 (이것도 Colab 터져요)

In [11]:
def create_sequences(values, time_steps=89):
    output = []
    for i in range(len(values) - time_steps):
        output.append(values[i : (i + time_steps)])
    # Convert 2D sequences into 3D as we will be feeding this into
    # a convolutional layer.
    return np.expand_dims(output, axis=2)

In [12]:
x_train = create_sequences(train_np)

In [16]:
x_train.shape

(215912, 89, 1, 79)

In [19]:
x_train = x_train.reshape((x_train.shape[0],x_train.shape[1],x_train.shape[3]))

In [20]:
x_train.shape

(215912, 89, 79)

# Conv1D

- 1D 컨브넷을 RNN 이전의 전처리 단계로 사용하면, 시퀀스 길이를 줄이고 RNN이 처리할 유용한 표현을 추출해준다
- 2D 컨브넷이 2D공간의 시각적 패턴을 잘 처리하는 것처럼, 1D 컨브넷은 시간에 따른 패턴을 잘 처리한다

긴 시퀀스 -> 1D CNN -> 짧은 시퀀스 -> RNN

# Timeseries anomaly detection using an Autoencoder

https://keras.io/examples/timeseries/timeseries_anomaly_detection/

input shape, output shape = (batch_size, timestep, num_features)으로 같다
여기서는 Conv1D를 사용.


In [28]:
model1 = Sequential(
    [
        layers.Input(shape=(x_train.shape[1], x_train.shape[2])),
        layers.Conv1D(
            filters=32, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        layers.Dropout(rate=0.2),
        layers.Conv1D(
            filters=16, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        layers.Conv1DTranspose(
            filters=16, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        layers.Dropout(rate=0.2),
        layers.Conv1DTranspose(
            filters=32, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        layers.Conv1DTranspose(filters=79, kernel_size=7, padding="same"),
    ]
)
model1.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss="mse")
model1.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_8 (Conv1D)            (None, 45, 32)            17728     
_________________________________________________________________
dropout_8 (Dropout)          (None, 45, 32)            0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 23, 16)            3600      
_________________________________________________________________
conv1d_transpose_12 (Conv1DT (None, 46, 16)            1808      
_________________________________________________________________
dropout_9 (Dropout)          (None, 46, 16)            0         
_________________________________________________________________
conv1d_transpose_13 (Conv1DT (None, 92, 32)            3616      
_________________________________________________________________
conv1d_transpose_14 (Conv1DT (None, 92, 79)           

In [29]:
history = model1.fit(x_train,
                    x_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.1)

KeyboardInterrupt: 

# Time Series Anomaly Detection with LSTM Autoencoders using Keras in Python

https://www.curiousily.com/posts/anomaly-detection-in-time-series-with-lstms-using-keras-in-python/

https://towardsdatascience.com/step-by-step-understanding-lstm-autoencoder-layers-ffab055b6352

Autoencoder를 이용한 Anomaly Dectection의 basic steps

1. 평범한 data (no anomalies)로 Autoencoder 학습
2. 새로운 data를 받아서 Autoencoder로 reconstruct
3. reconstruct한 data의 error 값이 특정 임계점을 넘으면, anomaly라고 정함

시계열 데이터의 temporal properties를 위해 LSTM을 추가사용

Our Autoencoder should take a sequence as input and outputs a sequence of the same shape.
Here’s how to build such a simple model in Keras:

# 층 요약
encoder: LSTM_1(128, return_sequences=True)
         LSTM_2(64, return_sequences=False)
         
RepeatVector_layer

decoder: LSTM_3(64, return_sequences=True)
         LSTM_4(128, return_sequences=False)
         TimeDistributed(Dense(79))


## Encoder
#### return_sequences=True: LSTM layer가 sequence를 만듬.

return_suquences = True 이면 LSTM에서 매 cell이 output을 내놓음.
원래 input = (batchsize, timestep, features) 이면

True인 LSTM   -> output = (batchsize, timestep, cell갯수) 
False인 LSTM  -> output = (batchsize, cell갯수)

LSTM_2를 지나면 (batchsize, cell 갯수)로 encoding 됨

##  RepeatVector layer: 단순히 input을 n번 반복함

RepeatVector layer를 통해, encoding된 정보가 단순히 n개 복사되어서
output = (batchsize, n, cell)로 늘어남. 
encoder와 decoder를 이어주는 다리 역할 (만약에 다음 layer가 단순히 Dense 라면, 이 RepeatVector layer 필요없음)

## Decoder
#### Encoder의 역순

LSTM_3 -> output=(batchsize, n, cell)
LSTM_3 -> output=(batchsize, n, cell)

####  TimeDistributed layer: 전 layer의 outputs의 숫자와 같은 길이의 벡터를 생성.
전의 output이 (n,cell)인데 여기서 (cell, 1)의 벡터를 생성하고 79번 곱한다.
그러면 output이 맨처음 input이랑 형태가 같아진다.


In [ ]:
model3 = Sequential()

model3.add(LSTM(128, activation='relu',
                input_shape=(ts,79), return_sequences=True))
model3.add(LSTM(64, activation='relu', return_sequences=False))
## encoder
model3.add(RepeatVector(ts))
model3.add(LSTM(64, activation='relu', return_sequences=True))
model3.add(LSTM(128, activation='relu', return_sequences=True)) #90 * 128
model3.add(TimeDistributed(Dense(79))) #128 * 79 -> 90 * 79


model3.compile(optimizer='adam', loss='mse')
model3.summary()

# Anomaly Detection Using VAE

https://data-newbie.tistory.com/100 https://thingsolver.com/time-series-anomaly-detection-using-a-variational-autoencoder-vae/

1. vae는 특정 값이 아니라 분포를 모델링한다.
2. anomalies란 결국 다른 처리로 인해 기존 분포와는 다른 것에서 나오는 존재이다.
3. 그러므로 high reconstruction error / low reconstruction으로 이러한 것을 잡아낼 수 있다는 것이다

기타 references

https://dacon.io/competitions/official/235624/data/


https://hoya012.github.io/blog/anomaly-detection-overview-1/

https://towardsdatascience.com/a-keras-based-autoencoder-for-anomaly-detection-in-sequences-75337eaed0e5

VAE: https://data-newbie.tistory.com/100 
https://thingsolver.com/time-series-anomaly-detection-using-a-variational-autoencoder-vae/


GAN: https://jayhey.github.io/semi-supervised%20learning/2017/12/08/semisupervised_generative_models/